In [12]:
import pandas as pd

# Load the datasets
schools_df = pd.read_csv('schools_complete.csv')
students_df = pd.read_csv('students_complete.csv')


In [13]:
# Calculate metrics for district summary
total_schools = schools_df['School ID'].nunique()
total_students = students_df['Student ID'].nunique()
total_budget = schools_df['budget'].sum()
average_math_score = students_df['math_score'].mean()
average_reading_score = students_df['reading_score'].mean()
passing_math = students_df[students_df['math_score'] >= 70]['Student ID'].count() / total_students * 100
passing_reading = students_df[students_df['reading_score'] >= 70]['Student ID'].count() / total_students * 100
overall_passing = students_df[(students_df['math_score'] >= 70) & (students_df['reading_score'] >= 70)]['Student ID'].count() / total_students * 100

# Create district summary DataFrame
district_summary_df = pd.DataFrame({
    "Total Schools": [total_schools],
    "Total Students": [total_students],
    "Total Budget": [total_budget],
    "Average Math Score": [average_math_score],
    "Average Reading Score": [average_reading_score],
    "% Passing Math": [passing_math],
    "% Passing Reading": [passing_reading],
    "% Overall Passing": [overall_passing]
})


In [14]:
# Merge the school and student data
school_data_complete = pd.merge(students_df, schools_df, how="left", on=["school_name", "school_name"])

# Group by school
school_group = school_data_complete.groupby(['school_name'])

# Calculate metrics
school_type = schools_df.set_index('school_name')['type']
total_students_school = school_group['Student ID'].count()
total_school_budget = schools_df.set_index('school_name')['budget']
per_student_budget = total_school_budget / total_students_school
average_math_score_school = school_group['math_score'].mean()
average_reading_score_school = school_group['reading_score'].mean()

# Calculate passing percentages
passing_math_school = school_data_complete[school_data_complete['math_score'] >= 70].groupby('school_name')['Student ID'].count() / total_students_school * 100
passing_reading_school = school_data_complete[school_data_complete['reading_score'] >= 70].groupby('school_name')['Student ID'].count() / total_students_school * 100
overall_passing_school = school_data_complete[(school_data_complete['math_score'] >= 70) & (school_data_complete['reading_score'] >= 70)].groupby('school_name')['Student ID'].count() / total_students_school * 100

# Create school summary DataFrame
school_summary_df = pd.DataFrame({
    "School Type": school_type,
    "Total Students": total_students_school,
    "Total School Budget": total_school_budget,
    "Per Student Budget": per_student_budget,
    "Average Math Score": average_math_score_school,
    "Average Reading Score": average_reading_score_school,
    "% Passing Math": passing_math_school,
    "% Passing Reading": passing_reading_school,
    "% Overall Passing": overall_passing_school
})


In [15]:
# Top 5 schools
top_schools = school_summary_df.sort_values("% Overall Passing", ascending=False).head(5)

# Bottom 5 schools
bottom_schools = school_summary_df.sort_values("% Overall Passing").head(5)


In [17]:
# Calculate average math score by grade for each school
math_scores_by_grade = school_data_complete.groupby(["school_name", "grade"]).math_score.mean().unstack()

# Calculate average reading score by grade for each school
reading_scores_by_grade = school_data_complete.groupby(["school_name", "grade"]).reading_score.mean().unstack()

# Rearrange the columns to display grades in order
math_scores_by_grade = math_scores_by_grade[['9th', '10th', '11th', '12th']]
reading_scores_by_grade = reading_scores_by_grade[['9th', '10th', '11th', '12th']]

# Display the DataFrame for math scores by grade
print("Math Scores by Grade:")
display(math_scores_by_grade)

# Display the DataFrame for reading scores by grade
print("\nReading Scores by Grade:")
display(reading_scores_by_grade)


Math Scores by Grade:


grade,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248



Reading Scores by Grade:


grade,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


In [19]:
import pandas as pd

# Define spending bins and labels
spending_bins = [0, 585, 630, 645, 680]
labels = ["<$585", "$585-630", "$630-645", "$645-680"]

# Categorize spending based on the bins
school_summary_df["Spending Ranges (Per Student)"] = pd.cut(school_summary_df["Per Student Budget"], spending_bins, labels=labels)

# Group by spending range while specifying observed=True for future compatibility
spending_math_scores = school_summary_df.groupby(["Spending Ranges (Per Student)"], observed=True)["Average Math Score"].mean()
spending_reading_scores = school_summary_df.groupby(["Spending Ranges (Per Student)"], observed=True)["Average Reading Score"].mean()
spending_passing_math = school_summary_df.groupby(["Spending Ranges (Per Student)"], observed=True)["% Passing Math"].mean()
spending_passing_reading = school_summary_df.groupby(["Spending Ranges (Per Student)"], observed=True)["% Passing Reading"].mean()
overall_passing_spending = school_summary_df.groupby(["Spending Ranges (Per Student)"], observed=True)["% Overall Passing"].mean()

# Create spending summary DataFrame
spending_summary_df = pd.DataFrame({
  "Average Math Score": spending_math_scores,
  "Average Reading Score": spending_reading_scores,
  "% Passing Math": spending_passing_math,
  "% Passing Reading": spending_passing_reading,
  "% Overall Passing": overall_passing_spending
})

# Display the spending summary DataFrame
display(spending_summary_df)


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$585,83.455399,83.933814,93.460096,96.610877,90.369459
$585-630,81.899826,83.155286,87.133538,92.718205,81.418596
$630-645,78.518855,81.624473,73.484209,84.391793,62.857656
$645-680,76.997210,81.027843,66.164813,81.133951,53.526855


In [21]:
import pandas as pd

# Assuming school_summary_df is already created with necessary columns

# Define size bins and labels for school size
size_bins = [0, 1000, 2000, 5000]
labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

# Categorize schools based on size bins
school_summary_df["School Size"] = pd.cut(school_summary_df["Total Students"], size_bins, labels=labels)

# Group by school size
size_math_scores = school_summary_df.groupby(["School Size"], observed=True)["Average Math Score"].mean()
size_reading_scores = school_summary_df.groupby(["School Size"], observed=True)["Average Reading Score"].mean()
size_passing_math = school_summary_df.groupby(["School Size"], observed=True)["% Passing Math"].mean()
size_passing_reading = school_summary_df.groupby(["School Size"], observed=True)["% Passing Reading"].mean()
overall_passing_size = school_summary_df.groupby(["School Size"], observed=True)["% Overall Passing"].mean()

# Create size summary DataFrame
size_summary_df = pd.DataFrame({
  "Average Math Score": size_math_scores,
  "Average Reading Score": size_reading_scores,
  "% Passing Math": size_passing_math,
  "% Passing Reading": size_passing_reading,
  "% Overall Passing": overall_passing_size
})


In [23]:
# Group by school type
type_math_scores = school_summary_df.groupby(["School Type"], observed=True)["Average Math Score"].mean()
type_reading_scores = school_summary_df.groupby(["School Type"], observed=True)["Average Reading Score"].mean()
type_passing_math = school_summary_df.groupby(["School Type"], observed=True)["% Passing Math"].mean()
type_passing_reading = school_summary_df.groupby(["School Type"], observed=True)["% Passing Reading"].mean()
overall_passing_type = school_summary_df.groupby(["School Type"], observed=True)["% Overall Passing"].mean()

# Create type summary DataFrame
type_summary_df = pd.DataFrame({
  "Average Math Score": type_math_scores,
  "Average Reading Score": type_reading_scores,
  "% Passing Math": type_passing_math,
  "% Passing Reading": type_passing_reading,
  "% Overall Passing": overall_passing_type
})
